In [ ]:
# George Miloshevich 2022
# The point of this notebook is to show that the new loss and the old loss give the same result
# This routine is written for two parameters: input folder for VAE weights and the given epoch. It shows us how good the reconstruction of the VAE works
import os, sys
import shutil
from pathlib import Path
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'  # https://stackoverflow.com/questions/65907365/tensorflow-not-creating-xla-devices-tf-xla-enable-xla-devices-not-set
import logging
from colorama import Fore # support colored output in terminal
from colorama import Style
if __name__ == '__main__':
    logger = logging.getLogger()
    logger.handlers = [logging.StreamHandler(sys.stdout)]
else:
    logger = logging.getLogger(__name__)
logger.level = logging.INFO

fold_folder = Path('./xforanalogs/tests/testskip2/fold_0')  # The name of the folder where the weights have been stored
checkpoint = 15       # The checkpoint at which the weights have been stored

import importlib.util
def module_from_file(module_name, file_path): #The code that imports the file which originated the training with all the instructions
            spec = importlib.util.spec_from_file_location(module_name, file_path)
            module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(module)
            return module
logger.info(f"{Fore.BLUE}") #  indicates we are inside the routine       
logger.info(f"{fold_folder = }")
logger.info(f"loading module from  {fold_folder.parent}/Funs.py")
from importlib import import_module
#foo = import_module(fold_folder+'/Funs.py', package=None)
foo = module_from_file("foo", f'{fold_folder.parent}/Funs.py')
ef = foo.ef # Inherit ERA_Fields_New from the file we are calling
ln = foo.ln
ut = foo.ut

run_vae_kwargs = ut.json2dict(f"{fold_folder.parent}/config.json")

logger.info("==Importing tensorflow packages===")
import random as rd  
from scipy.stats import norm
import numpy as np


if len(sys.argv) > 3:
    rd.seed(a=int(sys.argv[3]))
else:
    rd.seed(a=None) # None = system time

tff = foo.tff # tensorflow routines 
ut = foo.ut # utilities
logger.info("==Checking GPU==")
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

logger.info("==Checking CUDA==")
tf.test.is_built_with_cuda()

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy.mpl.geoaxes

import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeat
data_proj = ccrs.PlateCarree()

#from tensorflow.keras.preprocessing.image import ImageDataGenerator
sys.path.insert(1, '../ERA')
import cartopy_plots as cplt


logger.info("==Reading data==")

year_permutation = np.load(f'{fold_folder.parent}/year_permutation.npy')
i = int(np.load(f'{fold_folder}/fold_num.npy'))

time_start = ut.extract_nested(run_vae_kwargs, 'time_start')
time_end = ut.extract_nested(run_vae_kwargs, 'time_end')
T = ut.extract_nested(run_vae_kwargs, 'T')
if (ut.keys_exists(run_vae_kwargs, 'label_period_start') and ut.keys_exists(run_vae_kwargs, 'label_period_end')):
    label_period_start = ut.extract_nested(run_vae_kwargs, 'label_period_start')
    label_period_end = ut.extract_nested(run_vae_kwargs, 'label_period_end')
    if (label_period_start is not None) and (label_period_end is not None):
        summer_days = label_period_end - label_period_start - T + 1
    elif (label_period_start is None) and (label_period_end is not None):
        summer_days = label_period_end - time_start - T + 1
    elif (label_period_start is not None) and (label_period_end is None):
        summer_days = time_end - label_period_start - T + 1
    else:
        summer_days = time_end - time_start - T + 1
else:
    summer_days = time_end - time_start - T + 1
if ut.keys_exists(run_vae_kwargs, 'normalization_mode'):
    normalization_mode = ut.extract_nested(run_vae_kwargs, 'normalization_mode')
else:
    normalization_mode = None

if ut.keys_exists(run_vae_kwargs, 'keep_dims'):
    keep_dims = ut.extract_nested(run_vae_kwargs, 'keep_dims')
else:
    keep_dims = None
#X, lat, lon, vae, Z_DIM, N_EPOCHS, INITIAL_EPOCH, BATCH_SIZE, LEARNING_RATE, checkpoint_path, fold_folder, myinput, history = foo.PrepareDataAndVAE(fold_folder, DIFFERENT_YEARS=year_permutation[:800])
year_permutation_va = np.load(f'{fold_folder}/year_permutation_va.npy')
# Select times we want to show for reconstruction
if True: # select at random 10 years out of the validation set 
    year_permutation = list(year_permutation_va[rd.sample(range(len(year_permutation_va)), 10)])  
    day_permutation = rd.sample(range(summer_days*len(year_permutation)), 5) 
else: # avoid random permutation, just select minimum number of years allowed in fold
    year_permutation = [year_permutation_va[0]]
    day_permutation = range(23,23+5)##range(12,12+5)#[4,5,6,7,8] # the length has to be 5 for plotting purposes
    
#TODO: convert the day permuation to the appropriate day and year
logger.info(f"{year_permutation = },{day_permutation = }")

logger.info(f"{Style.RESET_ALL}")
run_vae_kwargs = ut.set_values_recursive(run_vae_kwargs, {'myinput' : 'N', 'year_permutation' :year_permutation})
if not os.path.exists(ut.extract_nested(run_vae_kwargs, 'mylocal')): # we are assuming that training was not run on R740server5
    run_vae_kwargs = ut.set_values_recursive(run_vae_kwargs, {'mylocal' : '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/PLASIM/'})
logger.info(f"{run_vae_kwargs = }")

history, N_EPOCHS, INITIAL_EPOCH, checkpoint_path, LAT, LON, vae, X_va, Y_va, X_tr, Y_tr, _ = foo.run_vae(fold_folder, **run_vae_kwargs)



In [2]:
X_va.shape, np.max(X_va), np.min(X_va)

((1050, 24, 48, 1), 0.9874537, 0.0014628773)

In [26]:
data = X_va[0:2,...]
z_mean, z_log_var, z, zz = vae.call_encoder_classifier(data)
reconstruction = vae.decoder(z)
reconstruction.shape

TensorShape([2, 24, 48, 1])

In [30]:
data = X_va[0:2,...]
factor = vae.k1*vae.encoder_input_shape[1]*vae.encoder_input_shape[2]
vae.coef_out*factor*tf.reduce_mean([tf.reduce_mean(vae.rec_loss_form(data[...,i][..., np.newaxis], reconstruction[...,i][..., np.newaxis])) for i in range(reconstruction.shape[3])] ) 

<tf.Tensor: shape=(), dtype=float32, numpy=12066.404>

In [32]:
vae.k1*tf.reduce_mean(
                tf.reduce_sum(
                    tf.keras.losses.binary_crossentropy(data, reconstruction, from_logits=vae.from_logits), axis=(1, 2) # -Y_n Log( P_n) - (1 - Y_n) Log( 1 - P_n) is the expression for binary entropy
                )
            )

<tf.Tensor: shape=(), dtype=float32, numpy=12066.404>